In [ ]:
import random
import json
import pandas as pd

In [ ]:
df = pd.read_excel(r"\\wwg00m.rootdom.net\DFS\HOME\FHEL7SE\Tanishq\lala\Hugging_Model\OSC\annotations_training.xlsx")

In [ ]:
# Open the file with explicit encoding parameter
with open("OSC/result.json", "r", encoding="utf-8") as f:
    data = json.load(f)


# Load the KPI mapping file
kpi_mapping = pd.read_csv("OSC/kpi_mapping.csv")
kpi_mapping = kpi_mapping.iloc[:, :5]
kpi_mapping.head()

In [ ]:
# Initialize lists to store data for new DataFrame
question_list = []
paragraph_list = []
label_list = []

for i in range(df.shape[0]):
    paragraph = df["relevant_paragraphs"][i].strip("[]").strip('""')
    kpi_id = df["kpi_id"][i]

    for j in range(kpi_mapping.shape[0]):
        if kpi_id == kpi_mapping["kpi_id"][j]:
            question = kpi_mapping["question"][j]
            LABEL = 1
            break

    # Append values to lists
    question_list.append(question)
    paragraph_list.append(paragraph)
    label_list.append(LABEL)

# Create new DataFrame
new_df = pd.DataFrame({"question": question_list, "paragraph": paragraph_list, "label": label_list})

# Display new DataFrame
print(new_df)

### genereate negative data

In [ ]:
ran_ques = []
ran_par = []
ran_lab = []


for i in range(df.shape[0]):
    paragraph = df["relevant_paragraphs"][i].strip("[]").strip('""')
    kpi_id = df["kpi_id"][i]

    for j in range(kpi_mapping.shape[0]):
        if kpi_id != kpi_mapping["kpi_id"][j]:
            question = kpi_mapping["question"][j]
            LABEL = 0

            # Append values to lists
            ran_ques.append(question)
            ran_par.append(paragraph)
            ran_lab.append(LABEL)


combined = list(zip(ran_ques, ran_par, ran_lab))
random.shuffle(combined)
sampled = random.sample(combined, 20)
ran_ques, ran_par, ran_lab = zip(*sampled)

sampled_df = pd.DataFrame({"question": ran_ques, "paragraph": ran_par, "label": ran_lab})

# Append the sampled DataFrame to new_df
new_df = new_df.append(sampled_df, ignore_index=True)

# Display new_df
print(new_df)

In [ ]:
# shuffling the data
shuffled_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
shuffled_df.to_csv("data/train_data.csv", index=False)